In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.python.client import device_lib

In [2]:
#device_lib.list_local_devices()

In [3]:
dir = os.path.dirname(os.path.realpath('__file__'))
print(dir)

/home/jgrietens/Documents/smartbeat/dl-track/2-0-simplified


In [4]:
tf.reset_default_graph()

### Model definition

In [9]:
#Project constants
NR_CLASSES = 6

#Reset graph
tf.reset_default_graph()

#Control the graph being used by creating and using the "with" command. 
graph1 = tf.Graph()
graph1.as_default()
    
with tf.name_scope("placeholders"):
    x = tf.placeholder(tf.float32, (None, 200, 3))
    y = tf.placeholder(tf.float32, (None, 6))
    lr = tf.placeholder(tf.float32)
          
with tf.name_scope("global_step"):
    # Creates a variable to hold the global_step. Global step is a variable that is updated every time training is started. 
    global_step = tf.Variable(0, trainable=False)
              
with tf.name_scope("layers"):
    #Add any layers that constitute the network.
    x_flat = tf.contrib.layers.flatten(x)
    l1 = tf.contrib.layers.fully_connected(x_flat, num_outputs = 100)
    l2 = tf.contrib.layers.fully_connected(l1, num_outputs = 50)
    l3 = tf.contrib.layers.fully_connected(l2, num_outputs = 20)
    
    #Layer before softmax
    logits = tf.contrib.layers.fully_connected(l3, num_outputs = NR_CLASSES) #layer before softmax. 
    
    #Prediction layer
    y_pred = tf.nn.softmax(logits, axis = 1)#last layer like a softmax. 
    
with tf.name_scope("loss"):
    #Define the loss function that needs to be minimized. 
    entropy = tf.losses.softmax_cross_entropy(onehot_labels= y,logits= logits,reduction=tf.losses.Reduction.SUM_BY_NONZERO_WEIGHTS) #if redctuion is None shape is [batch_size], otherwise it is reduced to a scalar. i.e. sum over all the samples.
    l = tf.reduce_sum(entropy) #combined intropy of samples in batch. 
        
with tf.name_scope("optimizer"):
    # define the an optimizer object.
    train_op = tf.train.AdamOptimizer(lr).minimize(l, global_step=global_step) #global step counts the ammount of training epochs that has happened. 
        
with tf.name_scope("accuracy"):
    #Value usefull when comparing probability vector with one-hot label : [0; 0.30; 0.70] vs [0; 0; 1]
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_pred, 1)) # check if largest value of label and prediction are the same. 
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) # calculate accuracy.
        
        
with tf.name_scope("summary"):
    #Summary nodes( values that we want tot track ) 
    tf.summary.scalar('learning_rate', lr) 
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('loss', l)
    tf.summary.scalar('global_step', global_step)
        
    #Creates an object that needs to be called in sess.run() to run all summary nodes. 
    create_summary_op = tf.summary.merge_all()
        
    #Write log to file for tensorboard. Start tensorboard by: tensorboard --logdir=path/to/log-director
    logdir = dir + "/temp/" + run_name + "/tb_log " ####### create variable for log file and graph)
    filewriter = tf.summary.FileWriter(logdir,graph1)
        
with tf.name_scope("initializer"):
    #Create an initializer object to be called when running.
    #If you are programming in the low-level TensorFlow API (that is, you are explicitly creating your own graphs and sessions), you must explicitly initialize the variables. Most high-level frameworks such as tf.contrib.slim, tf.estimator.Estimator and Keras automatically initialize variables for you before training a model.
    init_op = tf.global_variables_initializer()
        
with tf.name_scope("saver"):
    #Creates a saver object.
    saver = tf.train.Saver() 

NameError: name 'logdir' is not defined

### Training/running definition

In [ ]:
#Session
"""sess.run([fetch],{feed dict}): the fetch list determines the subgraphs to run in this session. The feeddict maps python datatypes to tf.tensors."""
def model_train(x_train, y_train, batch_size, nr_epochs, learning_rate, run_name, x_test = None, y_test = None):
    
    #Deterine nr. of batches in provided dataset. 
    nr_samples = len(x_train)
    nr_batches_in_dataset = nr_samples // batch_size
    
    with tf.Session() as sess:
        
        #Initialize weigths: last checkpoint or init_op.
        """Note: Shouldn't this whole if statement be integrated in init_op & the graph?"""
        checkpoint = tf.train.get_checkpoint_state("/temp/" + run_name )
        
        if checkpoint is None:
            print("No checkpoint was found, random initilization is started...")
            sess.run([init_op])
        else:
            saver.restore(sess, tf.train.latest_checkpoint(dir + "/temp/" + run_name + "/model"))
        
        #Go through all the different epochs
        for epoch_count in range(1,nr_epochs+1): 
            pos = 0
            print("EPOCH ",epoch_count)
            
            for step in range(1,nr_batches_in_dataset+1):

                #Create batch data
                x_batch= x_train[pos:pos+batch_size]
                y_batch= y_train[pos:pos+batch_size]
                
                #Training
                feed_dict = {x: x_batch, y: y_batch, lr: learning_rate} 
                _,summary,loss,acc=sess.run([train_op, create_summary_op, l, accuracy],feed_dict)
                print("-batch %d, minibatch_loss %f" %(step, loss))
                
                #Write summary to disk
                filewriter.add_summary(summary, global_step)
                
                #Increment to capture next batch
                pos = pos + batch_size
            
            # Test on test dataset, if provided, at end of epoch. 
            if x_test is not None and y_test is not None: 
                feed_dict = {x: x_test, y: y_test} 
                epoch_test_acc=sess.run([accuracy],feed_dict)
                print("Test Set Accuracy:", epoch_test_acc)
            else: 
                print("No testing data provided.")

        save_path = saver.save(sess, dir + "/temp/" + run_name + "/model", global_step= global_step, write_meta_graph=False)

        print("Weights saved in path: %s"  % dir + "/temp/" + run_name + "/model")


        # Print the global step. 
        print('global_step: %s' % tf.train.global_step(sess, global_step))

    
        
def model_predict(x_eval, graph = tf.get_default_graph()):
        
    with tf.Session() as sess:
        
        # Restore last saved checkpoint. 
        saver.restore(sess, tf.train.latest_checkpoint(dir + dir + "/temp/" + run_name + "/model"))
        
        print("Model restored.")
        
        #Evaluate(infer) the function on given input
        y_eval = sess.run([y_pred], feed_dict = {x: x_eval})
        
        return y_eval



### Test model

In [6]:
#toy dataset: 
k = np.transpose(np.array([range(0,200),range(0,200),range(0,200)]))
p = np.array(range(0,6))

x_train = np.array([k,k])
y_train =  np.array([p,p])
x_test = np.array([k,k])
y_test = np.array([p,p])

In [7]:
#toy input parameters
weight_save_path = os.getcwd()
batch_size = 2
nr_epochs = 5
learning_rate = 0.01

In [8]:
model_train(x_train, y_train, batch_size = batch_size, nr_epochs=nr_epochs, learning_rate = learning_rate, run_name= "1", x_test = x_test, y_test = y_test)

NameError: name 'model_train' is not defined